# Classification bio-informatique du virus de l’Hépatite C (HCV) : Génotype 1 vs Génotype 2

## Problématique
Le virus de l’hépatite C (HCV) présente une forte diversité génétique, structurée en **génotypes**.  
En pratique clinique, le génotypage est important pour la surveillance épidémiologique et, historiquement, pour guider certaines stratégies thérapeutiques.

**Objectif :** construire et évaluer des modèles de Machine Learning capables de distinguer **HCV génotype 1** vs **HCV génotype 2** à partir de séquences nucléotidiques (FASTA), en suivant une démarche d’évaluation robuste.

## Approche
1. Charger des séquences FASTA pour G1 et G2 (upload Colab).
2. Nettoyer les séquences (A/C/G/T uniquement) et filtrer par longueur (≈ génome complet).
3. Représenter les séquences par **k-mers** (n-grammes de nucléotides) + **TF‑IDF**.
4. Évaluer les modèles avec **StratifiedKFold (validation croisée)**.
5. Produire des métriques cliniquement interprétables : **ROC/AUC**, matrice de confusion, précision, rappel, F1.

## Données attendues
Uploadez **deux fichiers FASTA** :
- `hcv_g1.fasta` : HCV génotype 1
- `hcv_g2.fasta` : HCV génotype 2

> Astuce : assurez-vous d’avoir un nombre raisonnable de séquences dans chaque groupe (ex : 80–300).  
> Si les classes sont déséquilibrées, ce notebook applique un **sous-échantillonnage** pour équilibrer (bonne pratique).

In [ ]:
# =========================
# 0) Installation & imports
# =========================
!pip -q install biopython joblib

import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import files
from Bio import SeqIO

from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, classification_report,
    confusion_matrix, roc_curve, roc_auc_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1) Upload des FASTA (Google Colab)
Uploadez vos fichiers FASTA. Le notebook tente de détecter automatiquement les noms contenant `g1` et `g2` (ou `genotype1`, `genotype2`, `gt1`, `gt2`).  
Si vos fichiers ont d’autres noms, renommez-les avant upload.

In [ ]:
# =========================
# 1) Upload FASTA
# =========================
uploaded = files.upload()

keys = list(uploaded.keys())
low = [k.lower() for k in keys]

g1_candidates = [keys[i] for i,k in enumerate(low) if ("g1" in k) or ("genotype1" in k) or ("gt1" in k)]
g2_candidates = [keys[i] for i,k in enumerate(low) if ("g2" in k) or ("genotype2" in k) or ("gt2" in k)]

if not g1_candidates or not g2_candidates:
    raise FileNotFoundError(
        "Détection automatique échouée.\n"
        "Uploadez deux FASTA dont les noms contiennent 'g1' et 'g2' "
        "(ex: hcv_g1.fasta et hcv_g2.fasta)."
    )

G1_PATH = g1_candidates[0]
G2_PATH = g2_candidates[0]

print("Fichiers détectés :")
print(" - Génotype 1:", G1_PATH)
print(" - Génotype 2:", G2_PATH)

## 2) Nettoyage et filtrage des séquences
- Majuscules, remplacement U→T
- Suppression des caractères non A/C/G/T
- Filtrage par longueur minimale

Pour HCV, un génome quasi complet est souvent proche de **~9600 nt**.  
Par défaut, on garde `>= 9000`. Ajustez si besoin.

In [ ]:
# =========================
# 2) Nettoyage + lecture FASTA
# =========================
def clean_nucleotides(seq: str) -> str:
    seq = seq.upper().replace("U", "T")
    seq = re.sub(r"[^ACGT]", "", seq)
    return seq

def load_fasta(path: str, min_len: int) -> list[str]:
    seqs = []
    for rec in SeqIO.parse(path, "fasta"):
        s = clean_nucleotides(str(rec.seq))
        if len(s) >= min_len:
            seqs.append(s)
    return seqs

MIN_LEN = 9000

g1_seqs = load_fasta(G1_PATH, min_len=MIN_LEN)
g2_seqs = load_fasta(G2_PATH, min_len=MIN_LEN)

print("Après filtrage :")
print("G1:", len(g1_seqs), "| G2:", len(g2_seqs))

def lengths_stats(seqs):
    lens = np.array([len(s) for s in seqs], dtype=int)
    return int(lens.min()), int(np.median(lens)), int(lens.max())

if len(g1_seqs) > 0:
    print("G1 lengths (min/median/max):", lengths_stats(g1_seqs))
if len(g2_seqs) > 0:
    print("G2 lengths (min/median/max):", lengths_stats(g2_seqs))

if len(g1_seqs) < 30 or len(g2_seqs) < 30:
    print("\n⚠️ Attention: peu de séquences après filtrage. "
          "Vous pouvez réduire MIN_LEN ou télécharger plus de séquences.")

## 3) Équilibrage des classes (bonne pratique)
Si l’une des classes est majoritaire, on **sous-échantillonne** la classe majoritaire pour obtenir un jeu de données équilibré.  
Cela évite les biais et stabilise les métriques (orientation clinique).

- Label `0` = HCV_G1
- Label `1` = HCV_G2

In [ ]:
# =========================
# 3) Équilibrage + dataset
# =========================
min_n = min(len(g1_seqs), len(g2_seqs))
if min_n == 0:
    raise ValueError("Aucune séquence disponible dans l'une des classes après filtrage.")

g1_seqs_bal = np.random.choice(g1_seqs, size=min_n, replace=False).tolist()
g2_seqs_bal = np.random.choice(g2_seqs, size=min_n, replace=False).tolist()

X = np.array(g1_seqs_bal + g2_seqs_bal, dtype=object)
y = np.array([0]*min_n + [1]*min_n, dtype=int)

print("Dataset équilibré :")
print(" - HCV_G1:", int((y==0).sum()))
print(" - HCV_G2:", int((y==1).sum()))
print("Total:", len(X))

## 4) Représentation par k-mers (k=4) + TF‑IDF
Les k-mers capturent des motifs locaux et des signatures génomiques.

Paramètres :
- `k = 4` (bon compromis)  
Vous pouvez tester k=3 ou k=5 si vous avez le temps.

In [ ]:
# =========================
# 4) Modèles (pipelines)
# =========================
K = 4

vect = CountVectorizer(analyzer="char", ngram_range=(K, K))
tfidf = TfidfTransformer()

logreg = LogisticRegression(max_iter=4000, random_state=RANDOM_STATE)
rf = RandomForestClassifier(n_estimators=500, random_state=RANDOM_STATE, n_jobs=-1)

# SVM linéaire calibré => proba => ROC/AUC
svm_cal = CalibratedClassifierCV(
    estimator=LinearSVC(random_state=RANDOM_STATE),
    method="sigmoid",
    cv=3
)

pipelines = {
    "LogisticRegression": Pipeline([("vect", vect), ("tfidf", tfidf), ("clf", logreg)]),
    "RandomForest": Pipeline([("vect", vect), ("tfidf", tfidf), ("clf", rf)]),
    "LinearSVC_Calibrated": Pipeline([("vect", vect), ("tfidf", tfidf), ("clf", svm_cal)]),
}

## 5) Évaluation robuste : StratifiedKFold + ROC/AUC
On utilise **StratifiedKFold (5 folds)** pour conserver le ratio des classes dans chaque pli.  
On calcule des probabilités **out-of-fold** (prédictions sur des données jamais vues pendant l’entraînement du pli) pour :
- AUC (ROC)
- courbe ROC
- matrice de confusion au seuil 0.5

> Convention : classe positive = `HCV_G2` (label 1).

In [ ]:
# =========================
# 5) Cross-validation + métriques
# =========================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def evaluate_model(name, pipe):
    # Probabilités out-of-fold
    proba = cross_val_predict(pipe, X, y, cv=skf, method="predict_proba")[:, 1]
    pred = (proba >= 0.5).astype(int)

    acc = accuracy_score(y, pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y, pred, average="binary", pos_label=1)
    auc = roc_auc_score(y, proba)
    cm = confusion_matrix(y, pred)
    report = classification_report(y, pred, target_names=["HCV_G1", "HCV_G2"])

    return {
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "auc": auc,
        "cm": cm,
        "proba": proba,
        "pred": pred,
        "estimator": pipe,
        "report": report
    }

results = []
for name, pipe in pipelines.items():
    r = evaluate_model(name, pipe)
    results.append(r)
    print(f"=== {name} ===")
    print(f"Accuracy: {r['accuracy']:.4f} | Precision: {r['precision']:.4f} | Recall: {r['recall']:.4f} | F1: {r['f1']:.4f} | AUC: {r['auc']:.4f}")
    print()

## 6) Courbes ROC (comparaison des modèles)

In [ ]:
# =========================
# 6) ROC curves
# =========================
plt.figure(figsize=(6,6))
for r in results:
    fpr, tpr, _ = roc_curve(y, r["proba"])
    plt.plot(fpr, tpr, label=f"{r['model']} (AUC={r['auc']:.3f})")

plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate (1 - Spécificité)")
plt.ylabel("True Positive Rate (Sensibilité)")
plt.title("ROC — HCV Génotype 1 vs Génotype 2")
plt.legend()
plt.grid(True)
plt.show()

## 7) Matrices de confusion + rapports de classification

In [ ]:
# =========================
# 7) Confusion matrices
# =========================
def plot_cm(cm, title):
    plt.figure(figsize=(4.5,4))
    plt.imshow(cm)
    plt.title(title)
    plt.xticks([0,1], ["Pred G1", "Pred G2"])
    plt.yticks([0,1], ["True G1", "True G2"])
    for i in range(2):
        for j in range(2):
            plt.text(j, i, cm[i, j], ha="center", va="center")
    plt.colorbar()
    plt.tight_layout()
    plt.show()

for r in results:
    print(r["model"])
    print(r["report"])
    plot_cm(r["cm"], f"Confusion Matrix - {r['model']}")

## 8) Tableau de comparaison (trié par AUC)
En biomédecine, l’AUC est un indicateur standard pour comparer des classifieurs.

In [ ]:
# =========================
# 8) Summary table
# =========================
summary = pd.DataFrame([
    {"Model": r["model"], "Accuracy": r["accuracy"], "Precision": r["precision"], "Recall": r["recall"], "F1": r["f1"], "AUC": r["auc"]}
    for r in results
]).sort_values(by="AUC", ascending=False).reset_index(drop=True)

summary

## 9) Sauvegarde et téléchargement des artefacts

Dans cette étape finale, nous sauvegardons les artefacts du modèle ayant obtenu
les meilleures performances moyennes en validation croisée.

Les fichiers exportés sont :
- **best_model.joblib** : modèle entraîné final (réentraîné sur toutes les données)
- **model_comparison.csv** : tableau récapitulatif des performances (Accuracy, AUC, etc.)

Ces fichiers sont ensuite téléchargés localement depuis Google Colab afin de
permettre leur réutilisation (déploiement, rapport, ou analyse ultérieure).


In [ ]:
# =========================
# 9) Save & Download Best Model (Colab)
# =========================
import joblib
from google.colab import files

# Sélection du meilleur modèle
best = summary.iloc[0]["Model"]
best_result = next(r for r in results if r["model"] == best)

best_estimator = best_result["estimator"]

# Réentraînement sur toutes les données
best_estimator.fit(X, y)

# Sauvegarde locale (runtime Colab)
joblib.dump(best_estimator, "best_model.joblib")
summary.to_csv("model_comparison.csv", index=False)

print("Saved locally:")
print("- best_model.joblib")
print("- model_comparison.csv")
print("Best model:", best)

# Téléchargement vers la machine locale
files.download("best_model.joblib")
files.download("model_comparison.csv")


## 10) Interprétation clinique + limites (à reprendre dans votre rapport)
### Interprétation
- **ROC/AUC** résume la capacité du modèle à séparer G1 et G2 indépendamment d’un seuil.
- La matrice de confusion aide à interpréter les faux positifs / faux négatifs.

### Pourquoi éviter les scores parfaits (1.00 partout) ?
Cela peut indiquer :
- fuite de données (doublons / séquences identiques entre classes),
- biais de collecte,
- séparation trop facile due à des artefacts (ex : régions différentes, qualité différente).

### Limites
- Données publiques parfois biaisées (pays, sous-types dominants, séquences partielles).
- Validation externe (autre base de données) idéale pour renforcer la crédibilité.